In [ ]:
import allel
import malariagen_data
import pandas as pd
import numpy as np

### Writing out haplotypes for phylogenetic analysis

First, lets set the genome position of our focus, and also set a position for a separate analysis of a downstream and upstream region, to compare it to. We can also set the size of the flanking region in bp, I set this to 10kb either side of the focus (same as Xavi rdl ms).  

In [ ]:
region = '2L:2,358,158-2,431,617'

region_names = ['focal', 'upstream', 'downstream']
regions = [2_422_652, 2_358_158, 2_431_617]

locus_name = 'vgsc'
contig = '2L'
flanking = 10_000
out_prefix = "../fastas/" 

sample_sets = ['1177-VO-ML-LEHMANN-VMF00004', '1177-VO-ML-LEHMANN-VMF00015',
       '1178-VO-UG-LAWNICZAK-VMF00025',
       '1188-VO-NIANG-NIEL-SN-2304-VMF00259',
       '1190-VO-GH-AMENGA-ETEGO-VMF00013',
       '1190-VO-GH-AMENGA-ETEGO-VMF00014',
       '1190-VO-GH-AMENGA-ETEGO-VMF00028',
       '1190-VO-GH-AMENGA-ETEGO-VMF00029',
       '1190-VO-GH-AMENGA-ETEGO-VMF00046',
       '1190-VO-GH-AMENGA-ETEGO-VMF00047',
       '1190-VO-GH-AMENGA-ETEGO-VMF00088',
       '1190-VO-GH-AMENGA-ETEGO-VMF00102',
       '1230-VO-MULTI-AYALA-AYDI-GA-2204',
       '1237-VO-BJ-DJOGBENOU-VMF00050', '1237-VO-BJ-DJOGBENOU-VMF00067',
       '1237-VO-BJ-DJOGBENOU-VMF00212', '1244-VO-GH-YAWSON-VMF00051',
       '1244-VO-GH-YAWSON-VMF00149', '1245-VO-CI-CONSTANT-VMF00054',
       '1246-VO-TZ-KABULA-VMF00185', '1246-VO-TZ-KABULA-VMF00197',
       '1253-VO-TG-DJOGBENOU-VMF00052', '1264-VO-CD-WATSENGA-VMF00161',
       '1264-VO-CD-WATSENGA-VMF00164', '1270-VO-MULTI-PAMGEN-VMF00232',
       '1273-VO-ZM-MULEBA-VMF00176', '1274-VO-KE-KAMAU-VMF00246',
       '1281-VO-CM-CHRISTOPHE-VMF00208', '1281-VO-CM-CHRISTOPHE-VMF00227',
       '1288-VO-UG-DONNELLY-VMF00168', '1288-VO-UG-DONNELLY-VMF00219',
       '1296-VO-BF-DIABATE-VMF00272', '1314-VO-BF-KIENTEGA-KIMA-BF-2104',
       '1315-VO-NG-OMITOLA-OMOL-NG-2008', '1323-VO-GM-NGWA-VMF00235',
       '1323-VO-GM-NGWA-VMF00242', '1326-VO-UG-KAYONDO-KAJO-UG-2203',
       '1329-VO-GA-CHRISTOPHE-VMF00228', '1330-VO-GN-LAMA-VMF00250',
       '1339-VO-GH-AMENGA-ETEGO-VMF00302', '1351-VO-SS-WEETMAN-VMF00282', 
        'AG1000G-AO', 'AG1000G-BF-A', 'AG1000G-BF-B', 'AG1000G-BF-C',
       'AG1000G-CD', 'AG1000G-CF', 'AG1000G-CI', 'AG1000G-CM-A',
       'AG1000G-CM-B', 'AG1000G-CM-C', 'AG1000G-FR', 'AG1000G-GA-A',
       'AG1000G-GH', 'AG1000G-GM-A', 'AG1000G-GM-B', 'AG1000G-GM-C',
       'AG1000G-GN-A', 'AG1000G-GN-B', 'AG1000G-GQ', 'AG1000G-GW',
       'AG1000G-KE', 'AG1000G-ML-A', 'AG1000G-ML-B', 'AG1000G-MW',
       'AG1000G-MZ', 'AG1000G-TZ', 'AG1000G-UG',
       'barron-2019', 'bergey-2019', 'campos-2021', 'crawford-2016',
       'fontaine-2015-rebuild', 'tennessen-2021', '1338-VO-NG-ADEDAPO-VMF00268']

Then lets select which cohorts we want to load, connect to Ag3() API and load some metadata.

In [ ]:
# Define region strings
region_dict = dict(zip(region_names, regions))

for k, v in region_dict.items():
    region_dict[k] = f'{contig}:{v-flanking}-{v+flanking}'

ag3 = malariagen_data.Ag3(results_cache="../results_cache/")

In [ ]:
df_samples = ag3.sample_metadata(sample_sets)
df_samples['taxon'].value_counts()

### Write haplotypes to FASTA

In this notebook, we will construct FASTA sequences (or a multi-sequence alignment) from haplotypes in selected cohorts of the ag3, plus some outgroups. We will use *An. merus*, *An. melas*, and *An. quadriannulatus* as outgroups. 

In [ ]:
def haps_to_fasta_eric(haplos, var_alleles):
    return(var_alleles[range(var_alleles.shape[0]), np.transpose(haplos)])


def haps_to_fasta_df(haps):
    """
    Take haplotype XArray and create pandas df of fasta sequences
    """
    # transform xarray into haplotype array
    haplos = allel.GenotypeArray(haps['call_genotype'].compute()).to_haplotypes().values
    # extract ref and alt alleles array
    var_alleles = haps['variant_allele'].compute().values.astype(str)
        
    seq_arr = haps_to_fasta_eric(haplos, var_alleles)
    seq_arr = np.vstack(seq_arr)
    
    # Make dataframe of all haplotype sequences for region
    fasta_df = pd.DataFrame(seq_arr)
    fasta_df.columns = haps['variant_position'].compute().values
    sample_ids = haps['sample_id'].compute().values
    fasta_df.loc[:, 'hap'] = ["> " + p for p in make_unique(np.repeat(sample_ids, 2))]
    cols = list(fasta_df)
    # move the column to head of list using index, pop and insert
    cols.insert(0, cols.pop(cols.index('hap')))
    fasta_df = fasta_df.loc[:, cols]
    return(fasta_df)

def remove_missing_invariant_fasta_df(fasta_df, remove_invariant=False):
    """
    Remove any columns in pandas dataframe that are missing genotypes ('.')
    """
    # missing_bool = fasta_df.apply(lambda x: any(x == b'.') , axis=0)
    # print(f"Removing {missing_bool.sum()} alleles with a missing call")
    # fasta_df = fasta_df.loc[:, ~missing_bool]
    fasta_df = fasta_df.set_index('hap')
    
    if remove_invariant:
        invariant_cols = fasta_df.nunique() <= 1
        print(f"Removing {invariant_cols.sum()} invariant SNPs")
        fasta_df = fasta_df.loc[:, ~invariant_cols]
        print(f"There are {fasta_df.shape[0]} haplotypes and {fasta_df.shape[1]} segregating haplotype calls")

    # More efficient string joining
    sequences = [''.join(row) for row in fasta_df.values]
    fasta_df = fasta_df.copy()  # De-fragment first
    fasta_df['seq'] = sequences
    
    return(fasta_df.reset_index()[['hap', 'seq']])


def make_unique(values):
    value_counts = {}
    unique_values = []
    
    for value in values:
        if value in value_counts:
            value_counts[value] += 1
            unique_values.append(f"{value}_{value_counts[value]}")
        else:
            value_counts[value] = 0
            unique_values.append(f"{value}_{value_counts[value]}")
    
    return np.array(unique_values)

For each region, convert the haps to fasta and write to file! 

In [ ]:
haps = {}
for name, region in region_dict.items():
    # Load haplotypes for region, find intersection with phase 1 outgroup data
    print(f"loading haplotypes | {name} | {region}")
    haps[name] = ag3.haplotypes(region=region, sample_sets=sample_sets, analysis='gamb_colu_arab')
    print(name, region, haps[name]['call_genotype'].shape)
    sample_ids = haps[name]['sample_id'].compute().values
    
    # for each haplotype, loop through SNPs and create a FASTA sequence array depending on alleles
    print(f"Converting haps to FASTA sequence | {name}")
    fasta_df = haps_to_fasta_df(haps[name])
    
    # remove missing alleles in the outgroups and invariant sites 
    fasta_df = remove_missing_invariant_fasta_df(fasta_df, remove_invariant=False)
    
    # write to csv with \n sep to make FASTA file
    fasta_df.to_csv(f"{out_prefix}/{locus_name}_{name}.fasta", sep="\n", index=False, header=False)
    print(f'Multiple alignment FASTA written \n')
    
    df_samples = ag3.sample_metadata().set_index('sample_id')
    df_samples = df_samples.loc[sample_ids, :].reset_index()
    
    # Create haplotype version with consecutive haplotypes per mosquito
    df_samples_haps = df_samples.loc[df_samples.index.repeat(2)].reset_index(drop=True)
    # Create the sample_id column to match the fasta files
    df_samples_haps['sample_id'] = make_unique(np.repeat(df_samples['sample_id'].values, 2))    
    
    # remove > and join with metadata for each pop, useful for plotting phylo trees metadata
    fasta_df.loc[:, 'hap'] = fasta_df['hap'].str.strip('> ')
    all_haps = pd.concat([fasta_df.reset_index(drop=True), df_samples_haps.reset_index(drop=True)], axis=1)
    all_haps.to_csv(f"{out_prefix}/{locus_name}_{name}.metadata.tsv", sep="\t", index=False, header=True)

## Running IQTree
You may prefer to do this outside of the jupyter notebook, it can take a while :) 

In [ ]:
#for name in names:
#   !iqtree -s {locus_name}_{name}.fasta -B 10000